TO DO

* refactor validation and test steps
* probably need the finbert model in training mode too when training?
* keeping LSTM for classification?
* needs some hyperparameter tuning
* modify any of the args: get_text_split(text, length=200, overlap=50, max_chunks=4)?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%pip install datasets
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor
import datetime
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import numpy as np
from statistics import mean
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from torch.optim import SGD

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Selected device is {}'.format(device))

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert = AutoModel.from_pretrained("ProsusAI/finbert").to(device)
# bert = AutoModel.from_pretrained("bert-base-uncased").to(device)

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# distilbert = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

Selected device is cuda


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

**SET THE MODEL**

In [7]:
model = finbert

In [8]:
# checkpoint_dir = os.path.join(os.getcwd(),'checkpoints')
checkpoint_dir = '/content/drive/MyDrive/COMP0087/checkpoints_finbert_titles'

# Data loading


In [9]:
dataset = pd.read_csv('/content/drive/MyDrive/COMP0087/cleaned_dataset2.csv')
dataset

,symbol,publish_time,title,body,returns,trend,cleaned_body
0,A,2023-05-24,"Agilent (A) Q2 Earnings Match Estimates, Reven...",Agilent Technologies A reported second-quarter...,-5.620333,decrease,Agilent Technologies reported second quarter f...
1,A,2023-06-02,Agilent Intelligent Mass Spectrometry Solution...,Self-aware solutions making mass spectrometry ...,1.995527,stable,Self aware solutions making mass spectrometry ...
2,A,2023-11-20,Agilent Reports Fourth-Quarter Fiscal Year 202...,Revenue at the high end of guidance and EPS ex...,7.344231,increase,Revenue at the high end of guidance and EPS ex...
3,AAL,2023-05-22,The Undoing of American and JetBlue’s Northeas...,American Airlines CEO Robert Isom (left) speak...,-3.171385,decrease,American Airlines CEO Robert Isom left speakin...
4,AAL,2023-05-25,American Airlines flags no earnings impact fro...,(Reuters) - American Airlines Group Inc's Chie...,4.420287,increase,Reuters American Airlines Group Inc Chief Fina...
...,...,...,...,...,...,...,...
3283,ZION,2023-08-07,Why Zions Bancorp Stock Surged 42% in July,Zions Bancorporation (NASDAQ: ZION) rewarded i...,-3.270037,decrease,Zions Bancorporation NASDAQ ZION rewarded inve...
3284,ZION,2023-11-14,Regional Bank Stocks Rise After Bond Yields Drop,One big winner from Tuesday’s surprise inflati...,8.101503,increase,One big winner from Tuesday surprise inflation...
3285,ZTS,2023-06-15,Zoetis Releases 2022 Sustainability Report to ...,$7.4 million in corporate giving invested in c...,3.972232,increase,million in corporate giving invested in commun...
3286,ZTS,2023-08-08,Zoetis (ZTS) Q2 2023 Earnings Call Transcript ...,"Hosting the call today is Steve Frank, vice pr...",5.119265,increase,Hosting the call today is Steve Frank vice pre...


In [45]:
dataset.sort_values(by = 'publish_time', inplace=True)
dataset.reset_index(inplace=True, drop=True)

In [11]:
def get_smaller_dataset(df, size=0.25):

  df = df.dropna()
  sss = StratifiedShuffleSplit(n_splits=1, test_size=size, random_state=42)
  X = df.drop('trend', axis=1)
  y = df['trend']

  for train_index, test_index in sss.split(X, y):
      X_sample, y_sample = X.iloc[test_index], y.iloc[test_index]

  sample_df = X_sample.assign(trend=y_sample.values)
  sample_df.reset_index(inplace=True, drop=True)

  return sample_df

In [46]:
df = dataset[['symbol', 'title', 'trend']]
df.rename(columns={'title' : 'body'}, inplace=True)
df.head()

,symbol,body,trend
0,CHD,Church & Dwight (CHD) to Report Q1 Earnings: W...,stable
1,EVRG,Evergy Files Kansas First Rate Review in Five ...,stable
2,SWKS,Why Investors Need to Take Advantage of These ...,decrease
3,SWKS,14 Best Income Stocks To Invest In,stable
4,STX,Seagate Technology Holdings plc (NASDAQ:STX) Q...,stable


In [13]:
# df = get_smaller_dataset(df)

# Tokenize data

In [47]:
def wrap_tokenizer(tokenizer, padding=True, truncation=True, return_tensors='pt', max_length=None):
    def tokenize(text):
        text = list(text)
        tokens = tokenizer(
            text,
            padding=padding,
            return_attention_mask=True,
            truncation=truncation,
            max_length=max_length,
            return_tensors=return_tensors
            )
        return tokens['input_ids'], tokens['attention_mask']
    return tokenize

In [48]:
# tokenize = wrap_tokenizer(tokenizer, padding=True, truncation = True, return_tensors=None)
# tokens = tokenize(df['body'])
# num_tokens = [len(x) for x in tokens]
# df['length'] = pd.Series(num_tokens)

In [49]:
# max_length = df['length'].unique().max()
# print("Max length of news body: ", max_length)
# max_tokens = 512
# for i in range(1,(max_length//max_tokens)):
#     num = sum(df['length']>i*max_tokens)
#     print(f"Number of text that have more than {i}*max_tokens is {num}")

In [50]:
# sns.histplot(df['length'])

In [51]:
# df['length'].mode()

In [52]:
def get_text_split(text, length=500, overlap=50, max_chunks=4):
    chunks = []

    words = text.split()
    n_words = len(words)

    n = max(1, min(max_chunks, (n_words - length) // (length - overlap) + 1))

    for i in range(n):
        start_idx = i * (length - overlap)
        end_idx = min(start_idx + length, n_words)

        chunk_words = words[start_idx:end_idx]

        chunk_text = " ".join(chunk_words)

        # If it's the last chunk and its length is less than 75% of the desired length, skip
        if i == n - 1 and len(chunk_words) < 0.75 * length and n > 1:
            continue

        chunks.append(chunk_text)

    return chunks

**ENCODE LABELS**

In [53]:
labels = ["increase", "stable", "decrease"]

label_encoder = LabelEncoder()

df['trend'] = label_encoder.fit_transform(df['trend'])

# Train-val-test split

In [54]:
def split_df(df):
    n_rows = len(df)

    df_train = df.iloc[:int(0.8*n_rows),:]
    df_val = df.iloc[int(0.8*n_rows):int(0.9*n_rows),:]
    df_test = df.iloc[int(0.9*n_rows):,:]

    return df_train, df_val, df_test

In [55]:
n_rows = len(df)
dfs_train, dfs_val, dfs_test = [],[],[]
gb = df.groupby('trend')
for x in gb.groups:
    group = gb.get_group(x)
    df_train, df_val, df_test = split_df(group)
    dfs_train.append(df_train)
    dfs_val.append(df_val)
    dfs_test.append(df_test)

df_train = pd.concat(dfs_train, ignore_index=True)

df_val = pd.concat(dfs_val, ignore_index=True)

df_test = pd.concat(dfs_test, ignore_index=True)

print(f'Number of training examples: {len(df_train)}')
print(f'Number of validation examples: {len(df_val)}')
print(f'Number of test examples: {len(df_test)}')

Number of training examples: 2630
Number of validation examples: 329
Number of test examples: 329


In [56]:
df_train.body = df_train.body.apply(lambda x: get_text_split(x))
df_val.body = df_val.body.apply(lambda x: get_text_split(x))
df_test.body = df_test.body.apply(lambda x: get_text_split(x))

In [57]:
df_train['n_chunks'] = df_train.body.apply(lambda x: len(x))
df_val['n_chunks'] = df_val.body.apply(lambda x: len(x))
df_test['n_chunks'] = df_test.body.apply(lambda x: len(x))

In [58]:
df_train.head()

,symbol,body,trend,n_chunks
0,SWKS,[Why Investors Need to Take Advantage of These...,0,1
1,CDAY,[Ceridian HCM (CDAY) Q1 Earnings Lag Estimates],0,1
2,RJF,"[Raymond James Financial, Inc. (NYSE:RJF) Q2 2...",0,1
3,CDAY,[Ceridian Hcm (CDAY) Q1 2023 Earnings Call Tra...,0,1
4,OGN,[Organon Reports Results for the First Quarter...,0,1


# Model

In [59]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# def set_lr(optimizer, lr):
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr


def save_to_disk(txt_path, values):
    if os.path.isfile(txt_path):
        os.remove(txt_path)
    with open(txt_path, "wb") as fp:
        pickle.dump(values, fp)
    return


# def load_from_disk(txt_path):
#     with open(txt_path, "rb") as f:
#         values =  pickle.load(f)
#     return values


def save_checkpoint(model, classifier, optimizer, logs, epoch):
    print('')
    print('Saving checkpoint...')
    state_dict = {
        'model': model.state_dict(),
        'classifier':classifier.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    torch.save(state_dict, os.path.join(checkpoint_dir, 'checkpoint_{}.pt'.format(epoch)))
    save_to_disk(os.path.join(checkpoint_dir, 'logs.txt'),logs)
    print(f'Checkpoint saved!')


def load_checkpoint(path, model, classifier, device, optimizer=None):
    pretrained_dict = torch.load(path ,map_location=torch.device(device))
    classifier.load_state_dict(pretrained_dict['classifier'])
    model.load_state_dict(pretrained_dict['model'])
    # if optimizer is not None:
    #     optimizer.load_state_dict(pretrained_dict['optimizer'])
    #     return xlmr, classifier, optimizer
    return model, classifier

In [60]:
class MyDataset(Dataset):
    def __init__(self,df):
        self.n_chunks = df['n_chunks'].to_list()
        self.X = df['body'].to_list()
        self.Y = df['trend']

    def __len__(self):
        return len(self.X)

    def __getitem__(self,index):
        return self.X[index], self.Y.iloc[index], self.n_chunks[index]

def collate_func(batch):
    X = [x[0] for x in batch]
    Y = torch.Tensor([x[1] for x in batch])
    c = [x[2] for x in batch]
    return [X,Y,c]

In [61]:
class Classifier(nn.Module):
    def __init__(self, lstm_size, emb_dim, out_dim, dropout):
        super().__init__()

        self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=lstm_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(in_features=lstm_size, out_features=out_dim)

    def forward(self, x, n_chunks):
        x = pad_sequence(x, batch_first=True, padding_value=0)
        x = pack_padded_sequence(input=x, lengths=n_chunks, batch_first=True, enforce_sorted=False)
        x, _ = self.lstm(x)
        x, _ = pad_packed_sequence(x, batch_first=True)
        x = x[:,-1,:]
        x = self.dropout(x)
        logit = self.linear(x)
        return logit

In [62]:
def train(train_loader, tokenize, model, classifier, optimizer, scheduler, logs, dropout=0.0):
    classifier.train()
    #model.train()
    model.eval()

    accuracy = []
    losses = []

    for text, target, n_chunks in train_loader:
        logs['lr'].append(get_lr(optimizer))

        target = target.long().to(device)

        flat_text = [item for sublist in text for item in sublist]

        input_ids, attention_masks = tokenize(flat_text)
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)

        with torch.no_grad():
          outputs = model(input_ids=input_ids,
                          attention_mask=attention_masks)

        # outputs = model(input_ids=input_ids,
        #                 attention_mask=attention_masks)

        embeddings = outputs.last_hidden_state

        #pooled_emb = torch.mean(embeddings, axis=1)
        pooled_emb = (embeddings * attention_masks[:, :, None]).sum(dim=1) / attention_masks[:, :, None].sum(dim=1)

        pooled_emb = nn.Dropout(dropout)(pooled_emb)
        x = [s for s in torch.split(pooled_emb, n_chunks, dim=0)]

        logits = classifier(x, n_chunks)

        loss = nn.CrossEntropyLoss()(input=logits, target=target)

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        _, predictions = torch.max(logits, 1)
        correct_predictions = torch.sum(predictions == target).item()
        acc = correct_predictions / target.size(0)
        accuracy.append(acc)
        losses.append(loss.item())

    return accuracy, losses

In [63]:
def val_step(val_loader, tokenize, model, classifier, device, is_test=False):

    classifier.eval()
    model.eval()

    accuracy = []
    losses = []

    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for text, target, n_chunks in val_loader:
            target = target.long().to(device)

            flat_text = [item for sublist in text for item in sublist]
            # tokens = tokenize(flat_text)
            # tokens = tokens.to(device)

            # outputs = model(tokens)

            input_ids, attention_masks = tokenize(flat_text)
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_masks)

            embeddings = outputs.last_hidden_state

            #pooled_emb = torch.mean(embeddings, axis=1)
            pooled_emb = (embeddings * attention_masks[:, :, None]).sum(dim=1) / attention_masks[:, :, None].sum(dim=1)

            x = [s for s in torch.split(pooled_emb, n_chunks, dim=0)]

            logits = classifier(x, n_chunks)

            loss = nn.CrossEntropyLoss()(input=logits, target=target)

            _, predictions = torch.max(logits, 1)
            correct_predictions = torch.sum(predictions == target).item()
            acc = correct_predictions / target.size(0)
            accuracy.append(acc)
            losses.append(loss.item())

            if is_test:
                all_predictions.append(predictions.cpu())
                all_targets.append(target.cpu())

    if is_test:
        all_predictions = torch.cat(all_predictions).numpy()
        all_targets = torch.cat(all_targets).numpy()
        return accuracy, losses, all_predictions, all_targets

    return accuracy, losses

# Sentiment Analysis

**HYPERPARAMETERS**

In [64]:
BATCH_SIZE = 2
EPOCHS = 10
LR = 1e-7
EMBEDDING_DIM = 768 # 768 for base and 1024 for large
LSTM_SIZE = 64
DROPOUT = 0.2
POOLED_EMB_DO = 0.3
OUT_DIM_LIN = 3

In [65]:
train_dataset = MyDataset(df_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_func)

val_dataset = MyDataset(df_val)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_func)

In [66]:
classifier = Classifier(lstm_size=LSTM_SIZE,
                        emb_dim=EMBEDDING_DIM,
                        out_dim=OUT_DIM_LIN,
                        dropout= DROPOUT).to(device)

tokenize = wrap_tokenizer(tokenizer)

# If fine-tuning the model (FinBert)
#params = list(model.parameters()) + list(classifier.parameters())

# If using the base FinBert
params = list(classifier.parameters())

optimizer = AdamW(params, lr=0.0001)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=len(train_loader) * 1,
    num_training_steps=len(train_loader) * EPOCHS)


**TRAINING & VALIDATION**

In [67]:
logs = {'train_acc':[],
        'train_loss':[],
        'val_acc':[],
        'val_loss':[],
        'lr': []}

best_val_loss = float('inf')
for epoch in range(EPOCHS):

    train_acc, train_loss = train(train_loader=train_loader,
                                  tokenize=tokenize,
                                  model=model,
                                  classifier=classifier,
                                  optimizer=optimizer,
                                  scheduler=scheduler,
                                  logs=logs,
                                  dropout=POOLED_EMB_DO)

    val_acc, val_loss = val_step(val_loader=val_loader,
                                 tokenize=tokenize,
                                 model = model,
                                 classifier = classifier,
                                 device=device
                                )

    if mean(val_loss) < best_val_loss:
        best_val_loss = mean(val_loss)
        save_checkpoint(model = model, classifier = classifier, optimizer=optimizer, logs=logs, epoch=epoch)


    logs['train_acc'] += train_acc
    logs['train_loss'] += train_loss
    logs['val_acc'] += val_acc
    logs['val_loss'] += val_loss

    # Save checkpoint after each epoch
    # save_checkpoint(model = model, classifier = classifier, optimizer=optimizer, logs=logs, epoch=epoch)

    print(f"Epoch {epoch} --> train_loss:{mean(train_loss):.4f},\
                              train_acc:{mean(train_acc): .2f}%, \
                              val_loss:{mean(val_loss): .4f}, \
                              val_acc:{mean(val_acc): .2f}%")



Saving checkpoint...
Checkpoint saved!
Epoch 0 --> train_loss:0.7459,                              train_acc: 0.67%,                               val_loss: 2.1132,                               val_acc: 0.40%

Saving checkpoint...
Checkpoint saved!
Epoch 1 --> train_loss:0.6980,                              train_acc: 0.74%,                               val_loss: 2.0957,                               val_acc: 0.40%

Saving checkpoint...
Checkpoint saved!
Epoch 2 --> train_loss:0.7540,                              train_acc: 0.71%,                               val_loss: 1.9834,                               val_acc: 0.40%

Saving checkpoint...
Checkpoint saved!
Epoch 3 --> train_loss:0.8130,                              train_acc: 0.67%,                               val_loss: 1.8667,                               val_acc: 0.40%

Saving checkpoint...
Checkpoint saved!
Epoch 4 --> train_loss:0.8754,                              train_acc: 0.63%,                               val_loss

**TESTING**

In [79]:
path = '/content/drive/MyDrive/COMP0087/checkpoints_finbert_full_dataset/checkpoint_9.pt'

In [80]:
model, classifier = load_checkpoint(path, model, classifier, device)

In [81]:
test_dataset = MyDataset(df_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_func)

In [82]:
test_accuracy, test_loss, all_predictions, all_targets = val_step(val_loader=test_loader,
                                                          tokenize=tokenize,
                                                          model = model,
                                                          classifier = classifier,
                                                          device=device,
                                                          is_test = True
                                                          )

In [83]:
print(f"Test loss:{mean(test_loss):.4f}\n test acc:{mean(test_accuracy):.2f}%")

Test loss:1.0552
 test acc:0.47%
